## Aula 5.2.Coleta de dados fundamentalistas

### Exemplo 1 - Coletando a lista de companhias com registro na CVM

Acessar o site https://www.rad.cvm.gov.br/ENET/frmConsultaExternaCVM.aspx

In [1]:
import requests_html
import re
import pandas as pd

In [2]:
import os
import sys
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
CLASSES_DIR = os.getenv("CLASSES_DIR")

sys.path.append(CLASSES_DIR)

In [3]:
from listar_empresas_registradas_cvm import EmpresasRegistradasCVM

reg_emp = EmpresasRegistradasCVM()
reg_emp.empresas.head()

,key,value,status
0,018554,"PAN FINANCEIRA S.A. - CRÉDITO, FINANCIAMENTO ...",REGISTRO ATIVO
1,252224,2W ECOBANK S.A.,NaN
2,025224,2W ECOBANK S.A.,REGISTRO ATIVO
3,917581,2W ENERGIA S.A.,REGISTRO ATIVO
4,021954,3A COMPANHIA SECURITIZADORA,REGISTRO CANCELADO


In [4]:
from listar_documentos_cvm import ListarDocumentosCVM

# Docs_CVM = ListarDocumentosCVM(['021610','025461'], '01/01/2021', '01/01/2023')

# Docs_CVM.obter_documentos()
# doc_cvm_res = Docs_CVM.cvt_response_to_df()

# doc_cvm_res.head()

In [5]:
from database.sqlite_db import DatabaseConnectionFactory

conn = DatabaseConnectionFactory.CreateDbConnection()
c = conn.cursor()

# c.execute("delete from documentos_cvm")
# conn.commit()

# for row, doc in doc_cvm_res.iterrows():
#     sql = f"REPLACE INTO documentos_cvm VALUES ('{doc['Cod_CVM']}', {doc['Sequencia']}, {doc['Versao']}, '{doc['Protocolo']}', '{doc['Tipo']}')"
#     c.execute(sql)

# conn.commit()

result = c.execute("select * from documentos_cvm").fetchall()
conn.close()

result

[('021610', 103505, 1, '021610ITR310320210100103505-72', 'ITR'),
 ('025461', 103710, 1, '025461ITR310320210100103710-75', 'ITR'),
 ('025461', 103735, 2, '025461ITR310320210200103735-68', 'ITR'),
 ('025461', 104287, 3, '025461ITR310320210300104287-72', 'ITR'),
 ('025461', 107406, 1, '025461ITR300620210100107406-78', 'ITR'),
 ('025461', 107427, 2, '025461ITR300620210200107427-72', 'ITR'),
 ('025461', 107438, 3, '025461ITR300620210300107438-76', 'ITR'),
 ('021610', 107516, 1, '021610ITR300620210100107516-76', 'ITR'),
 ('025461', 107530, 4, '025461ITR300620210400107530-79', 'ITR'),
 ('025461', 109616, 1, '025461ITR300920210100109616-84', 'ITR'),
 ('025461', 109702, 2, '025461ITR300920210200109702-71', 'ITR'),
 ('021610', 109787, 1, '021610ITR300920210100109787-89', 'ITR'),
 ('025461', 114673, 1, '025461ITR310320220100114673-74', 'ITR'),
 ('021610', 114979, 1, '021610ITR310320220100114979-74', 'ITR'),
 ('021610', 119633, 1, '021610ITR300620220100119633-78', 'ITR'),
 ('025461', 119796, 1, '0

### Exemplo 2 - Coleta de dados da CVM usando requests-html

No exemplo a seguir vamos primeiramente entender como o site de documentos da CVM funciona.

Vamos inicialmente inspecionar o site https://www.rad.cvm.gov.br/ENET/frmConsultaExternaCVM.aspx

In [6]:
from download_doc_xml_rad_cvm import DownloadDocXmlRadCvm
from database.Documento_CVM import Documento_CVM

#teste_doc = Documento_CVM(('021610', 103505, 1, '021610ITR310320210100103505-72', 'ITR'))
teste_doc = Documento_CVM(('025461', 119796, 1, '025461ITR300620220100119796-76', 'ITR'))
down_doc = DownloadDocXmlRadCvm(teste_doc)

In [7]:
my_dict = down_doc.dados

In [8]:
# my_dict#['Documento'].keys()

In [9]:
my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario'].keys()

dict_keys(['DadosEmpresa', 'DfIndividuais', 'DfConsolidadas', 'PareceresDeclaracoes'])

In [10]:
my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario']['DfConsolidadas']

{'BalancoPatrimonialAtivo': {'Conta': [{'ContaFixa': 'True',
    'CodigoConta': '1',
    'DescricaoConta': 'Ativo Total',
    'TrimestreAtual': '899.807',
    'ExercicioAnterior': '999.039'},
   {'ContaFixa': 'True',
    'CodigoConta': '1.01',
    'DescricaoConta': 'Ativo Circulante',
    'TrimestreAtual': '529.935',
    'ExercicioAnterior': '667.693'},
   {'ContaFixa': 'True',
    'CodigoConta': '1.01.01',
    'DescricaoConta': 'Caixa e Equivalentes de Caixa',
    'TrimestreAtual': '203.692',
    'ExercicioAnterior': '271.532'},
   {'ContaFixa': 'False',
    'CodigoConta': '1.01.01.01',
    'DescricaoConta': None,
    'TrimestreAtual': '0',
    'ExercicioAnterior': '0'},
   {'ContaFixa': 'False',
    'CodigoConta': '1.01.01.02',
    'DescricaoConta': None,
    'TrimestreAtual': '0',
    'ExercicioAnterior': '0'},
   {'ContaFixa': 'False',
    'CodigoConta': '1.01.01.03',
    'DescricaoConta': None,
    'TrimestreAtual': '0',
    'ExercicioAnterior': '0'},
   {'ContaFixa': 'False',
   

In [11]:
my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario']['DfConsolidadas'].keys()

dict_keys(['BalancoPatrimonialAtivo', 'BalancoPatrimonialPassivo', 'DemonstracaoResultado', 'DemonstracaoResultadoAbrangente', 'DemonstracaoFluxoCaixa', 'DemonstracaoMutacoesPatrimonioLiquido', 'DemonstracaoValorAdicionado'])

#### Balanço Patrimonial Ativo

In [12]:
doc = "BalancoPatrimonialAtivo"

In [13]:
my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario']['DfConsolidadas'][doc]

{'Conta': [{'ContaFixa': 'True',
   'CodigoConta': '1',
   'DescricaoConta': 'Ativo Total',
   'TrimestreAtual': '899.807',
   'ExercicioAnterior': '999.039'},
  {'ContaFixa': 'True',
   'CodigoConta': '1.01',
   'DescricaoConta': 'Ativo Circulante',
   'TrimestreAtual': '529.935',
   'ExercicioAnterior': '667.693'},
  {'ContaFixa': 'True',
   'CodigoConta': '1.01.01',
   'DescricaoConta': 'Caixa e Equivalentes de Caixa',
   'TrimestreAtual': '203.692',
   'ExercicioAnterior': '271.532'},
  {'ContaFixa': 'False',
   'CodigoConta': '1.01.01.01',
   'DescricaoConta': None,
   'TrimestreAtual': '0',
   'ExercicioAnterior': '0'},
  {'ContaFixa': 'False',
   'CodigoConta': '1.01.01.02',
   'DescricaoConta': None,
   'TrimestreAtual': '0',
   'ExercicioAnterior': '0'},
  {'ContaFixa': 'False',
   'CodigoConta': '1.01.01.03',
   'DescricaoConta': None,
   'TrimestreAtual': '0',
   'ExercicioAnterior': '0'},
  {'ContaFixa': 'False',
   'CodigoConta': '1.01.01.04',
   'DescricaoConta': None,
  

In [14]:
df_bpa = pd.DataFrame()
for row in my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario']['DfConsolidadas'][doc]['Conta']:
    df_bpa = pd.concat([df_bpa, pd.DataFrame([row])])

In [15]:
df_bpa

,ContaFixa,CodigoConta,DescricaoConta,TrimestreAtual,ExercicioAnterior
0,True,1,Ativo Total,899.807,999.039
0,True,1.01,Ativo Circulante,529.935,667.693
0,True,1.01.01,Caixa e Equivalentes de Caixa,203.692,271.532
0,False,1.01.01.01,None,0,0
0,False,1.01.01.02,None,0,0
...,...,...,...,...,...
0,False,1.02.04.02.16,None,0,0
0,False,1.02.04.02.17,None,0,0
0,False,1.02.04.02.18,None,0,0
0,False,1.02.04.02.19,None,0,0


#### Balanço Patrimonial Passivo

In [16]:
doc = "BalancoPatrimonialPassivo"

In [17]:
my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario']['DfConsolidadas'][doc]

{'Conta': [{'ContaFixa': 'True',
   'CodigoConta': '2',
   'DescricaoConta': 'Passivo Total',
   'TrimestreAtual': '899.807',
   'ExercicioAnterior': '999.039'},
  {'ContaFixa': 'True',
   'CodigoConta': '2.01',
   'DescricaoConta': 'Passivo Circulante',
   'TrimestreAtual': '143.399',
   'ExercicioAnterior': '187.188'},
  {'ContaFixa': 'True',
   'CodigoConta': '2.01.01',
   'DescricaoConta': 'Obrigações Sociais e Trabalhistas',
   'TrimestreAtual': '10.983',
   'ExercicioAnterior': '25.463'},
  {'ContaFixa': 'True',
   'CodigoConta': '2.01.01.01',
   'DescricaoConta': 'Obrigações Sociais',
   'TrimestreAtual': '10.983',
   'ExercicioAnterior': '25.463'},
  {'ContaFixa': 'False',
   'CodigoConta': '2.01.01.01.01',
   'DescricaoConta': 'Obrigações Sociais e Trabalhistas',
   'TrimestreAtual': '10.983',
   'ExercicioAnterior': '25.463'},
  {'ContaFixa': 'False',
   'CodigoConta': '2.01.01.01.02',
   'DescricaoConta': None,
   'TrimestreAtual': '0',
   'ExercicioAnterior': '0'},
  {'Cont

In [18]:
df_bpp = pd.DataFrame()
for row in my_dict['XmlInformacoesTrimestraisFinanceiras']['DadosITR']['Formulario']['DfConsolidadas'][doc]['Conta']:
    df_bpp = pd.concat([df_bpp, pd.DataFrame([row])])

In [19]:
df_bpp

,ContaFixa,CodigoConta,DescricaoConta,TrimestreAtual,ExercicioAnterior
0,True,2,Passivo Total,899.807,999.039
0,True,2.01,Passivo Circulante,143.399,187.188
0,True,2.01.01,Obrigações Sociais e Trabalhistas,10.983,25.463
0,True,2.01.01.01,Obrigações Sociais,10.983,25.463
0,False,2.01.01.01.01,Obrigações Sociais e Trabalhistas,10.983,25.463
...,...,...,...,...,...
0,False,2.03.08.17,None,0,0
0,False,2.03.08.18,None,0,0
0,False,2.03.08.19,None,0,0
0,False,2.03.08.20,None,0,0
